# Extended Subgraph Partitioning (orb-models)

In [2]:
import numpy as np
import torch

from tqdm import tqdm

from partitioner import part_graph_extended
import networkx as nx

## Partitioning Atoms

Loading a sample atomic dataset and converting it into a graph

In [3]:
from ase.io import read
from orb_models.forcefield.atomic_system import ase_atoms_to_atom_graphs
from ase.build import make_supercell
 
atoms = read("datasets/H2O.xyz")
atoms = make_supercell(atoms, [[1, 0, 0], [0, 1, 0], [0, 0, 1]])

# Instead of using neighborlist, I use the ase_atoms_to_atom_graphs provided by orb. Hopefully this will provide better results
atom_graph = ase_atoms_to_atom_graphs(atoms) # Keep this to compare results later

senders = atom_graph.senders
receivers = atom_graph.receivers
edge_feats = atom_graph.edge_features

G = nx.Graph()
G.add_nodes_from(range(len(atoms)))

for i, u in enumerate(senders):
    G.add_edge(u.item(), receivers[i].item(), weight=edge_feats['r'])

print("Number of atoms", len(atoms))

Number of atoms 192


Partition the computational graph into the number of desired partitions with the specified neighborhood distance

In [11]:
desired_partitions = 20
neighborhood_distance = 4
partitions, extended_partitions = part_graph_extended(G, desired_partitions, neighborhood_distance)

num_partitions = len(partitions)

print(f"Created {num_partitions} partitions")
print(f"Average partition size: {sum(len(x) for x in extended_partitions) / num_partitions}")

Created 20 partitions
Average partition size: 426.0


Create the ASE atoms object for each partition

In [12]:
from ase import Atoms

partitioned_atoms = []
indices_map = [] # Table mapping each atom in each partition back to its index in the original atoms object

for part in extended_partitions:
    current_partition = []
    current_indices_map = []
    for atom_index in part:
        current_partition.append(atoms[atom_index])
        current_indices_map.append(atoms[atom_index].index)

    partitioned_atoms.append(Atoms(current_partition, cell=atoms.cell, pbc=atoms.pbc)) # It's important to pass atoms.cell and atoms.pbc here
    indices_map.append(current_indices_map)

In [13]:
reconstructed_atoms = []
for atom_index in range(len(atoms)):
    reconstructed_atoms.append(atoms[atom_index])
reconstructed_atoms = Atoms(reconstructed_atoms, cell=atoms.cell, pbc=atoms.pbc)

reconstructed_atoms

Atoms(symbols='C110H276Ga8S8Si24', pbc=True, cell=[[11.548332214355469, 6.6674418449401855, 11.98120403289795], [-11.54824447631836, 6.667413234710693, 11.981135368347168], [1.9159999283147044e-05, -13.334798812866211, 11.981139183044434]])

## Inference

In [14]:
from orb_models.forcefield import atomic_system, pretrained
from orb_models.forcefield import segment_ops

Load model

In [5]:
from orb_models.forcefield.pretrained import get_base, load_model_for_inference
from orb_models.forcefield.graph_regressor import GraphRegressor, EnergyHead, NodeHead, GraphHead

device = "cpu"  # or device="cuda"

base = get_base(num_message_passing_steps=4)

model = GraphRegressor(
    graph_head=EnergyHead(
        latent_dim=256,
        num_mlp_layers=1,
        mlp_hidden_dim=256,
        target="energy",
        node_aggregation="mean",
        reference_energy_name="vasp-shifted",
        train_reference=True,
        predict_atom_avg=True,
    ),
    node_head=NodeHead(
        latent_dim=256,
        num_mlp_layers=1,
        mlp_hidden_dim=256,
        target="forces",
        remove_mean=True,
    ),
    stress_head=GraphHead(
        latent_dim=256,
        num_mlp_layers=1,
        mlp_hidden_dim=256,
        target="stress",
        compute_stress=True,
    ),
    model=base,
)

orbff = load_model_for_inference(model, 'https://orbitalmaterials-public-models.s3.us-west-1.amazonaws.com/forcefields/orb-v2-20241011.ckpt', device)

/opt/anaconda3/envs/orb-partitioning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/orb-partitioning/lib/python3.10/site-packages/orb_models/forcefield/pretrained.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user vi

Run inference on each partition

In [25]:
aggregated_features = torch.zeros((len(atoms), 256), dtype=torch.float32, device=device)

for i, part in tqdm(enumerate(partitioned_atoms), total=num_partitions):
    input_graph = atomic_system.ase_atoms_to_atom_graphs(part)

    batch = orbff.model(input_graph)

    feat = batch.node_features["feat"]

    for j, node in enumerate(part):
        original_index = indices_map[i][j]
        if original_index in partitions[i]: # If the node is a root node of the partition
            aggregated_features[original_index] = feat[j]

aggregated_features.shape

100%|██████████| 20/20 [00:06<00:00,  3.05it/s]


torch.Size([426, 256])

## Prediction

Run the prediction on the original, unpartitioned graph to obtain a benchmark for our results

In [9]:
result = orbff.predict(atom_graph)
benchmark_energy = result["graph_pred"]
benchmark_forces = result["node_pred"]
benchmark_stress = result["stress_pred"]
benchmark_energy

tensor([-972.0552])

In [18]:
from orb_models.forcefield.graph_regressor import ScalarNormalizer, LinearReferenceEnergy
from orb_models.forcefield.reference_energies import REFERENCE_ENERGIES

ref = REFERENCE_ENERGIES["vasp-shifted"]
reference = LinearReferenceEnergy(
    weight_init=ref.coefficients, trainable=True
)

n_node = torch.tensor([aggregated_features.shape[0]])

### Energy

In [19]:
input = segment_ops.aggregate_nodes(
    aggregated_features,
    n_node,
    reduction="mean"
)

energy = orbff.graph_head.mlp(input)
energy = orbff.graph_head.normalizer.inverse(energy).squeeze(-1)
energy = energy * n_node
energy = energy + reference(atom_graph.atomic_numbers, atom_graph.n_node)
energy

tensor([-2246.8350], grad_fn=<AddBackward0>)

In [20]:
print(f"Absolute error: {torch.abs(benchmark_energy - energy).item()}")
print(f"Percent error: {torch.abs((benchmark_energy - energy) / benchmark_energy).item() * 100}%")
print(f"Maximum error: {torch.max(benchmark_energy - energy).item()}")

Absolute error: 0.0
Percent error: 0.0%
Maximum error: 0.0


### Forces

In [21]:
forces = orbff.node_head.mlp(aggregated_features)
system_means = segment_ops.aggregate_nodes(
    forces, n_node, reduction="mean"
)
node_broadcasted_means = torch.repeat_interleave(
    system_means, n_node, dim=0
)
forces = forces - node_broadcasted_means
forces = orbff.node_head.normalizer.inverse(forces)
forces

tensor([[-0.0688,  0.0882,  0.2663],
        [ 0.0824,  0.1349,  0.2663],
        [ 0.0129, -0.0198,  0.2584],
        ...,
        [ 0.0539, -0.0196,  0.1435],
        [-0.0231,  0.0474,  0.1652],
        [ 0.0963,  0.0796,  0.1533]])

In [22]:
mae = torch.mean(torch.abs(benchmark_forces - forces))
mape = 100 * torch.mean(torch.abs((benchmark_forces - forces) / benchmark_forces))
max = torch.max(torch.abs(benchmark_forces - forces))

print(f"Mean absolute error: {mae.item()}")
print(f"Mean absolute percent error: {mape}%")
print(f"Maximum error: {max}")

Mean absolute error: 0.0
Mean absolute percent error: 0.0%
Maximum error: 0.0


### Stress

In [23]:
input = segment_ops.aggregate_nodes(
    aggregated_features,
    n_node,
    reduction="mean",
)
stress = orbff.stress_head.mlp(input)
stress = stress.squeeze(-1)
stress = orbff.stress_head.output_activation(stress)
stress = orbff.stress_head.normalizer.inverse(stress)
stress

tensor([[-1.4797e-02, -1.7188e-02, -1.2462e-02,  2.4029e-05,  3.1426e-05,
          5.2789e-07]])

In [24]:
mae = torch.mean(torch.abs(benchmark_stress - stress))
mape = 100 * torch.mean(torch.abs((benchmark_stress - stress) / benchmark_stress))
max = torch.max(torch.abs(benchmark_stress - stress))
print(f"Mean absolute error: {mae.item()}")
print(f"Mean absolute percent error: {mape}%")
print(f"Max error: {max}")

Mean absolute error: 0.0
Mean absolute percent error: 0.0%
Max error: 0.0
